In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# PCA + classification on RNAseq data 
# source: https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

In [37]:
# read data 
data = pd.read_csv('data_RNAseq.csv', index_col=0)
data.head()

,gene_0,gene_1,gene_2,gene_3,gene_4,gene_5,gene_6,gene_7,gene_8,gene_9,...,gene_20521,gene_20522,gene_20523,gene_20524,gene_20525,gene_20526,gene_20527,gene_20528,gene_20529,gene_20530
sample_0,0.0,2.017209,3.265527,5.478487,10.431999,0.0,7.175175,0.591871,0.0,0.0,...,4.926711,8.210257,9.723516,7.220030,9.119813,12.003135,9.650743,8.921326,5.286759,0.0
sample_1,0.0,0.592732,1.588421,7.586157,9.623011,0.0,6.816049,0.000000,0.0,0.0,...,4.593372,7.323865,9.740931,6.256586,8.381612,12.674552,10.517059,9.397854,2.094168,0.0
sample_2,0.0,3.511759,4.327199,6.881787,9.870730,0.0,6.972130,0.452595,0.0,0.0,...,5.125213,8.127123,10.908640,5.401607,9.911597,9.045255,9.788359,10.090470,1.683023,0.0
sample_3,0.0,3.663618,4.507649,6.659068,10.196184,0.0,7.843375,0.434882,0.0,0.0,...,6.076566,8.792959,10.141520,8.942805,9.601208,11.392682,9.694814,9.684365,3.292001,0.0
sample_4,0.0,2.655741,2.821547,6.539454,9.738265,0.0,6.566967,0.360982,0.0,0.0,...,5.996032,8.891425,10.373790,7.181162,9.846910,11.922439,9.217749,9.461191,5.110372,0.0


In [38]:
# get labels 
labels = pd.read_csv('labels_RNAseq.csv', index_col = 0)
labels

,Class
sample_0,PRAD
sample_1,LUAD
sample_2,PRAD
sample_3,PRAD
sample_4,BRCA
...,...
sample_796,BRCA
sample_797,LUAD
sample_798,COAD
sample_799,PRAD


In [39]:
# het numeric labels, to do: matrix labels 
labels['Class'].unique()
labels['Class'] = labels['Class'].astype('category').cat.codes
labels

,Class
sample_0,4
sample_1,3
sample_2,4
sample_3,4
sample_4,0
...,...
sample_796,0
sample_797,3
sample_798,1
sample_799,4


In [129]:
from sklearn.model_selection import train_test_split

# split the data in to 20% testset and 80% training set
train, test, train_label, test_label = train_test_split( data, labels, test_size=1/5, random_state=0)

In [131]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# Fit on training set only.
scaler.fit(train)

# scale data using standard deviation
train_no = scaler.transform(train)
test_no = scaler.transform(test)

In [132]:
from sklearn.decomposition import PCA
# Make an instance of the Model 
# scikit-learn choose the minimum number of principal components such that 95% of the variance is retained
pca = PCA(.95)

In [133]:
# use training set to set hyperparameters 
pca.fit(train_no)

PCA(n_components=0.95)

In [134]:
# number of PC that explains 95% of the data 
pca.n_components_

434

In [135]:
# dimension reduction to pca.n_component_ 
# to do: dont use traing hyperparameters for the test set
train = pca.transform(train_no)
test = pca.transform(test_no)

In [136]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [137]:
# train data using logisticregression
logisticRegr = LogisticRegression()
logisticRegr_no_pca = LogisticRegression()

# train with linear classifier (worksheet 2) 
linear = SVC(kernel='linear')

In [138]:
# train the model 
logisticRegr.fit(train, train_label)
linear.fit(train, train_label)

logisticRegr_no_pca.fit(train_no, train_label)

C:\Users\yoric\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\yoric\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\yoric\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  r

LogisticRegression()

In [139]:
# Prediction for One sample 
logisticRegr.predict(test[0].reshape(1,-1))

array([0], dtype=int8)

In [141]:
# prediction for first 10 samples 
print(logisticRegr.predict(test[0:10]))
print(linear.predict(test[0:10]))
print(logisticRegr_no_pca.predict(test_no[0:10]))

[0 0 4 0 0 0 3 0 2 0]
[0 0 4 0 0 0 3 0 2 0]
[0 0 4 0 0 0 3 0 2 0]


In [143]:
# score prediction 
print(logisticRegr.score(test, test_label))

from sklearn.metrics import accuracy_score
y_predicted = linear.predict(test)
print(accuracy_score(test_label, y_predicted))

1.0
1.0
